In [1]:
from gravpop import *

In [2]:
gwpop = GWPopLoader(posterior_file = "/Users/asadh/Documents/Data/posteriors.pkl",
                    prior_file = "/Users/asadh/Documents/Data/production.prior",
                    vt_file = "/Users/asadh/Downloads/o1+o2+o3_bbhpop_real+semianalytic-LIGO-T2100377-v2.hdf5",
                    enforce_minimum_neffective_per_event = "injection_resampling_vt")
gwpop.load_prior();
gwpop.load_model();
gwpop.load_vt();
gwpop.load_posteriors();
gwpop.create_likelihood();

11:00 bilby INFO    : Using SmoothedMassDistribution.
11:00 bilby INFO    : Created SmoothedMassDistribution with arguments {'mmin': 2, 'mmax': 100}
11:00 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
11:00 bilby INFO    : Created gwpopulation.models.redshift.PowerLawRedshift with arguments {'z_max': 1.9}
11:00 bilby INFO    : Using SmoothedMassDistribution.
11:00 bilby INFO    : Created SmoothedMassDistribution with arguments {'mmin': 2, 'mmax': 100}
11:00 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
11:00 bilby INFO    : Created gwpopulation.models.redshift.PowerLawRedshift with arguments {'z_max': 1.9}
11:00 bilby INFO    : Loading VT data from /Users/asadh/Downloads/o1+o2+o3_bbhpop_real+semianalytic-LIGO-T2100377-v2.hdf5.


In [71]:
from gwpopulation.models.redshift import PowerLawRedshift
from bilby.core.result import read_in_result
from scipy.interpolate import interp1d
import numpy as np
import deepdish as dd
import matplotlib.pyplot as plt

PP_path = '/Users/asadh/Documents/Data/analyses/PowerLawPeak/o1o2o3_mass_c_iid_mag_iid_tilt_powerlaw_redshift_result.json'
PP_result = read_in_result(PP_path)

PP_hyperposterior_samples = PP_result.posterior.copy() # making a copy is best practice here so you don't accidentally modify things in-place
PP_hyperposterior_samples

,alpha,beta,mmax,mmin,lam,mpp,sigpp,delta_m,mu_chi,sigma_chi,...,lamb,amax,log_likelihood,log_prior,selection,pdet_n_effective,surveyed_hypervolume,log_10_rate,rate,min_event_n_effective
0,3.476035,1.192781,91.137635,3.335027,0.045152,32.019619,7.361617,5.502380,0.430299,0.037676,...,0.593878,1.0,86.219165,-20.158271,0.001980,5600.884717,1102.923927,1.539321,34.619495,184.210871
1,2.843807,0.551632,92.336309,3.455594,0.099132,21.146326,9.635711,3.504334,0.229716,0.027052,...,3.899003,1.0,88.795439,-20.158271,0.000176,7628.305892,24548.963167,1.203419,15.974191,91.831207
2,4.127894,1.910420,99.382743,4.114949,0.058528,26.304442,9.105968,4.430823,0.297703,0.036759,...,-0.096742,1.0,89.598445,-20.158271,0.002577,4734.966457,617.064144,1.681428,48.020645,94.982019
3,4.512565,1.291055,98.845047,2.963035,0.007879,32.267134,9.914114,9.800366,0.378698,0.052658,...,3.511903,1.0,90.035214,-20.158271,0.000295,5654.797132,16703.390755,1.146385,14.008291,289.936772
4,2.772482,0.149371,94.491001,5.138160,0.035747,30.469008,8.569162,3.361789,0.336149,0.034270,...,2.514446,1.0,90.339131,-20.158271,0.000582,8247.427572,6336.065138,1.302851,20.084052,261.964627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11179,3.388853,1.075150,89.768520,4.684262,0.030388,35.727936,3.478996,5.246642,0.244086,0.027675,...,2.733047,1.0,107.871029,-20.158271,0.000525,3796.539977,7811.787060,1.235183,17.186329,71.639539
11180,3.684463,1.539779,81.785901,5.327478,0.030328,34.102957,3.505780,5.133147,0.251205,0.033394,...,3.171738,1.0,107.877236,-20.158271,0.000377,4006.532961,11954.783380,1.175202,14.969305,72.109121
11181,3.684463,1.539779,81.785901,5.327478,0.030328,34.102957,3.505780,5.133147,0.251205,0.033394,...,3.171738,1.0,107.877236,-20.158271,0.000377,4006.532961,11954.783380,1.169904,14.787819,72.109121
11182,3.573136,0.627634,88.689645,5.303211,0.029184,34.600418,3.531467,4.342366,0.229689,0.032670,...,3.456785,1.0,107.909312,-20.158271,0.000266,3745.787380,15818.169705,1.239911,17.374440,73.194989


In [72]:
cols = ['alpha','beta','mmax','mmin','lam','mpp','sigpp','delta_m','lamb']
param = PP_hyperposterior_samples[cols].iloc[0,:].to_dict()

gwpop.likelihood.parameters.update(param)
gwpop.likelihood.log_likelihood_ratio()

-1.7976931348623157e+308

In [75]:
gwpop.likelihood.parameters.update(param)
jnp.min(gwpop.likelihood.per_event_bayes_factors_and_n_effective_and_variances()[1])

Array(324.4489, dtype=float32)

In [5]:
mass_vars = ['alpha','beta','mmax','mmin','lam','mpp','sigpp','delta_m']
redshift_vars = ['lamb']

data = gwpop.likelihood.data

gwpop.likelihood.hyper_prior.models[0](data, **{k:v for k,v in param.items() if k in mass_vars})

array([[1.81946419e-02, 2.05055871e-02, 8.39644874e-03, ...,
        1.24354022e-02, 1.51105668e-02, 6.39469672e-03],
       [7.21595262e-03, 6.76803039e-03, 2.82002383e-01, ...,
        4.25122925e-03, 1.86779676e-01, 7.86951213e-03],
       [1.34137718e-02, 7.62464422e-03, 6.88927795e-03, ...,
        1.90680942e-02, 1.60372877e-02, 1.80551872e-02],
       ...,
       [9.71181743e-03, 1.11598984e-02, 1.60799699e-02, ...,
        1.11493628e-02, 1.59525600e-02, 1.32437229e-02],
       [5.94310159e-03, 9.23066772e-03, 5.28777518e-03, ...,
        5.74732560e-05, 1.18276542e-03, 4.37294227e-03],
       [1.84042269e-02, 1.31857000e-02, 1.30101194e-02, ...,
        4.62215738e-03, 8.08583980e-03, 3.41295212e-04]])

In [6]:
param2 = param.copy()
param2['sigpp'] = 1.0

In [12]:
data = gwpop.likelihood.data.copy()
data['prior'] = gwpop.likelihood.sampling_prior

In [13]:
import gravpop

SM = SmoothedTwoComponentPrimaryMassRatio(primary_mass_name='mass_1')
R = gravpop.PowerLawRedshift(z_max=1.9)

selection_data = {k:v for k,v in gwpop.selection.data.items()}

HL = PopulationLikelihood(models=[SM,R], 
                         event_data=data,
                         selection_data=selection_data,
                         analysis_time=selection_data['analysis_time'],
                         total_generated=selection_data['total_generated'])
#loglikes = HL.sampled_compute_log_weights(selection_data, param2)
#HL.compute_selection_N_eff(loglikes, N=HL.selection_data.total_generated)[0]
#HL.compute_selection_N_eff(loglikes)

In [77]:
HP

NameError: name 'HP' is not defined

In [14]:
aa = gwpop.selection.detection_efficiency(param2)
aa[0]**2 / aa[1]

3980.7235257324546

In [15]:
param3 = param2.copy()
param3['lam'] = 0.7
gwpop.likelihood.hyper_prior.parameters.update(param3)
gwpop.likelihood.hyper_prior._get_function_parameters(gwpop.likelihood.hyper_prior.models[0])

{'mpp': 32.01961882574348,
 'alpha': 3.476035201127745,
 'delta_m': 5.502380007170098,
 'mmin': 3.3350270645371465,
 'mmax': 91.13763545839001,
 'beta': 1.1927812704856988,
 'lam': 0.7,
 'sigpp': 1.0}

In [16]:
gwpop.likelihood.hyper_prior.prob(gwpop.likelihood.data) / gwpop.likelihood.sampling_prior

array([[3.71563614e+00, 3.62272878e-01, 3.54235391e-04, ...,
        1.72580650e-02, 1.87694364e-01, 2.39300490e-04],
       [2.02517014e-02, 2.02208607e-02, 1.38753518e+00, ...,
        1.24046758e-02, 8.39746952e-01, 2.36644036e-02],
       [1.94026722e+00, 4.99522627e-04, 4.78471456e-04, ...,
        4.28943170e-03, 5.26447224e+00, 7.96469814e-01],
       ...,
       [6.15355024e-05, 3.96908741e-03, 1.55051527e-04, ...,
        1.18182751e-04, 3.87426508e-01, 4.35388223e-02],
       [9.65964894e-06, 3.06704113e-05, 1.03129960e-05, ...,
        3.91120643e-06, 4.17631130e-06, 5.91948398e-06],
       [2.36815439e-01, 2.21666509e-03, 1.56500053e-03, ...,
        8.18230522e-05, 1.53469944e-04, 1.57758006e-05]])

In [19]:
gwpop.likelihood.hyper_prior.parameters.update(param2)
gwpop.likelihood.hyper_prior.models[0](gwpop.likelihood.data, **gwpop.likelihood.hyper_prior._get_function_parameters(gwpop.likelihood.hyper_prior.models[0]))

array([[1.12010654e-01, 1.36991214e-02, 6.56799418e-04, ...,
        1.51106515e-03, 4.74131967e-03, 5.27287030e-04],
       [7.02995368e-03, 6.55051144e-03, 2.81732178e-01, ...,
        4.04060950e-03, 1.86511621e-01, 7.65485152e-03],
       [4.14229042e-02, 5.99528987e-04, 5.52129283e-04, ...,
        3.86075058e-03, 1.07684514e-01, 1.44362384e-02],
       ...,
       [7.62483914e-04, 2.29716668e-03, 2.99385328e-03, ...,
        8.75516814e-04, 4.75705049e-02, 2.26553830e-02],
       [5.83067229e-04, 8.30573251e-04, 5.24800124e-04, ...,
        5.48342757e-05, 2.38250644e-04, 5.74924981e-04],
       [2.72045855e-02, 1.41262710e-03, 1.16132960e-03, ...,
        4.98834154e-04, 6.67044482e-04, 1.37018145e-04]])

In [20]:
HL.models[0](HL.event_data, param2)

Array([[1.1201065e-01, 1.3699093e-02, 6.5679953e-04, ..., 1.5110649e-03,
        4.7413148e-03, 5.2728702e-04],
       [7.0299748e-03, 6.5505351e-03, 2.8173578e-01, ..., 4.0406282e-03,
        1.8651988e-01, 7.6548769e-03],
       [4.1422922e-02, 5.9952919e-04, 5.5212923e-04, ..., 3.8607507e-03,
        1.0768453e-01, 1.4436220e-02],
       ...,
       [7.6248380e-04, 2.2971611e-03, 2.9938542e-03, ..., 8.7551714e-04,
        4.7570460e-02, 2.2655357e-02],
       [5.8306742e-04, 8.3057344e-04, 5.2480015e-04, ..., 5.4834287e-05,
        2.3825066e-04, 5.7492510e-04],
       [2.7204657e-02, 1.4126274e-03, 1.1613298e-03, ..., 4.9883430e-04,
        6.6704483e-04, 1.3701816e-04]], dtype=float32)

In [21]:
gwpop.likelihood.parameters.update(param2)
gwpop.likelihood.hyper_prior.prob(gwpop.likelihood.data, **param2)

array([[8.85424270e-03, 1.25064650e-03, 5.92749905e-05, ...,
        1.03774925e-04, 1.04443856e-04, 5.55856667e-05],
       [1.17136351e-04, 7.23507156e-05, 2.97600853e-03, ...,
        3.47438744e-05, 2.30413104e-03, 9.25075418e-05],
       [1.39044518e-03, 2.18480329e-05, 1.70780932e-05, ...,
        1.61237619e-04, 3.72980922e-03, 2.61483597e-04],
       ...,
       [3.38919815e-04, 1.20804254e-03, 1.79070805e-03, ...,
        2.96759651e-04, 1.57869879e-02, 1.34073290e-02],
       [4.03637078e-04, 4.78905244e-04, 3.51511678e-04, ...,
        2.04808771e-05, 1.59462971e-04, 4.23928886e-04],
       [8.00893145e-03, 5.30866192e-04, 2.92745761e-04, ...,
        1.36042078e-04, 1.48582743e-04, 4.22660206e-05]])

In [22]:
HL.models[0](gwpop.likelihood.data, param2) * HL.models[1](gwpop.likelihood.data, param2)

Array([[8.85415077e-03, 1.25064712e-03, 5.92755168e-05, ...,
        1.03774124e-04, 1.04445360e-04, 5.55860752e-05],
       [1.17146730e-04, 7.23456396e-05, 2.97558913e-03, ...,
        3.47504501e-05, 2.30393792e-03, 9.25166896e-05],
       [1.39040116e-03, 2.18474488e-05, 1.70775857e-05, ...,
        1.61237025e-04, 3.72968451e-03, 2.61501467e-04],
       ...,
       [3.38919839e-04, 1.20803982e-03, 1.79070851e-03, ...,
        2.96759768e-04, 1.57869775e-02, 1.34073095e-02],
       [4.03637328e-04, 4.78905538e-04, 3.51511728e-04, ...,
        2.04808839e-05, 1.59463016e-04, 4.23928926e-04],
       [8.00895412e-03, 5.30866324e-04, 2.92746001e-04, ...,
        1.36042101e-04, 1.48582985e-04, 4.22660232e-05]], dtype=float32)

In [23]:
#jnp.log(gwpop.likelihood.per_event_bayes_factors_and_n_effective()[0])
gwpop.likelihood.parameters.update(param2)
jnp.log(jnp.sum((gwpop.likelihood.hyper_prior.prob(gwpop.likelihood.data) / gwpop.likelihood.sampling_prior), axis=-1) / gwpop.likelihood.samples_per_posterior )

Array([ -1.9919136 ,   1.2934306 ,  -1.1601865 ,  -7.8853173 ,
        -1.6516606 ,   0.1774375 ,  -2.0637631 ,  -3.9015913 ,
        -2.8131306 ,  -1.9155035 ,  -1.6703235 ,  -2.2714005 ,
        -3.5980635 ,  -3.5531743 ,  -9.214896  ,  -6.17375   ,
        -9.367379  ,   0.45200068,  -7.569106  ,   0.53003126,
        -2.3585792 ,  -4.25607   , -10.18944   ,  -4.9288163 ,
        -2.7783163 ,  -4.080309  ,  -4.9802985 ,  -7.4817357 ,
        -1.8770456 ,  -4.009459  ,  -3.3439732 ,  -9.965057  ,
        -3.29976   ,  -3.4320524 ,  -4.682404  ,   0.23335826,
        -0.11446601,  -5.9797945 ,  -5.4353185 ,  -2.560652  ,
       -11.838063  ,  -8.565896  ,   0.31696725,   1.8574107 ,
        -7.718902  ,  -3.7707994 ,   0.5774144 ,   0.52590585,
         0.8139755 ,   0.27017716,  -2.0783064 ,   0.9477345 ,
        -3.2077818 ,  -8.479487  ,  -5.808269  ,  -5.5423317 ,
        -6.8291144 ,   1.5954273 ,  -3.724963  ,  -5.438617  ,
        -0.3764935 ,  -8.136147  ,  -1.6700652 ,  -1.40

In [24]:
HL.sampled_event_bayes_factors(HL.event_data, param2)

Array([ -1.9919138 ,   1.2934647 ,  -1.1601839 ,  -7.885317  ,
        -1.6516619 ,   0.1774559 ,  -2.0637622 ,  -3.9015903 ,
        -2.81313   ,  -1.9155025 ,  -1.6703205 ,  -2.2713995 ,
        -3.5980616 ,  -3.553173  ,  -9.214895  ,  -6.173749  ,
        -9.367378  ,   0.45204926,  -7.569105  ,   0.5300522 ,
        -2.3585787 ,  -4.256069  , -10.18944   ,  -4.928816  ,
        -2.7783155 ,  -4.080308  ,  -4.980298  ,  -7.481735  ,
        -1.8770442 ,  -4.0094585 ,  -3.3439717 ,  -9.965057  ,
        -3.2997594 ,  -3.4320517 ,  -4.682403  ,   0.23337555,
        -0.11444712,  -5.9797935 ,  -5.435318  ,  -2.5606503 ,
       -11.838062  ,  -8.565895  ,   0.316988  ,   1.8574162 ,
        -7.7189016 ,  -3.7707992 ,   0.5774498 ,   0.5259247 ,
         0.8139992 ,   0.27020264,  -2.0783057 ,   0.94778633,
        -3.2077813 ,  -8.479486  ,  -5.8082685 ,  -5.5423307 ,
        -6.829114  ,   1.5954771 ,  -3.7249618 ,  -5.4386163 ,
        -0.3764758 ,  -8.136146  ,  -1.6700644 ,  -1.40

In [ ]:
PopulationLikelihood.from_file()

In [26]:
{k:v for k,v in gwpop.selection.data.items()}

{'mass_1': array([17.39769745, 27.59049225, 34.87878799, ..., 45.73864544,
        51.90734739, 35.8333281 ]),
 'mass_ratio': array([0.89051247, 0.88072689, 0.98982847, ..., 0.98976494, 0.87312348,
        0.90983453]),
 'a_1': array([0.66609327, 0.77165711, 0.41970315, ..., 0.45309051, 0.00470902,
        0.48066029]),
 'a_2': array([0.84130038, 0.26786459, 0.2287109 , ..., 0.9919398 , 0.44948848,
        0.93711565]),
 'cos_tilt_1': array([-0.37940198, -0.17241716,  0.21326955, ..., -0.17223099,
        -0.56725815, -0.16667346]),
 'cos_tilt_2': array([ 0.38100872,  0.5378555 , -0.14621086, ..., -0.1658206 ,
         0.80703564, -0.1641033 ]),
 'redshift': array([0.19250949, 0.3339659 , 0.98206556, ..., 0.29160039, 0.12487546,
        0.33013271]),
 'prior': array([1.09751025e-04, 9.41494849e-05, 2.51845033e-04, ...,
        3.36005874e-05, 2.40199909e-06, 8.27588353e-05]),
 'total_generated': 73280198,
 'analysis_time': 2.000455579638502}

In [20]:
import inspect
inspect_it = lambda x: print(inspect.getsource(x))

In [33]:
inspect_it(HL.sampled_compute_log_weights)

    def sampled_compute_log_weights(self, data, params):
        return sum(self.log(model(data, params)) for model in self.models) - self.log(data["prior"])



In [96]:
inspect_it(gwpop.likelihood.per_event_bayes_factors_and_n_effective_and_variances)

    def per_event_bayes_factors_and_n_effective_and_variances(self):
        """
        Called by `_compute_per_event_ln_bayes_factors` to compute the
        per event BFs, effective number of samples for each event's computed
        BF, and the associated uncertainty (variance) in the *ln* BF. Computes
        same qunatities as superclass function `_compute_per_event_ln_bayes_factors`
        but additionally provides the effective sample size.

        Returns
        -------
        per_event_bfs: array-like
            The BF per event, computed by reweighting single-event likelihood
            samples into the `hyper_prior` model.
        n_effectives: array-like
            The effective sample size for each Monte Carlo sum computation of the BFs.
            The BF is computed for each event, so this array has length n_events.
        variance: array-like
            The variances (uncertainties) in the ln BF per event.
        """
        weights = self.hyper_prior.prob(se

In [40]:
def create_likes():
    gwpop = GWPopLoader(posterior_file = "/Users/asadh/Documents/Data/posteriors.pkl",
                        prior_file = "/Users/asadh/Documents/Data/production.prior",
                        vt_file = "/Users/asadh/Downloads/o1+o2+o3_bbhpop_real+semianalytic-LIGO-T2100377-v2.hdf5",
                        enforce_minimum_neffective_per_event = "injection_resampling_vt")
    gwpop.load_prior();
    gwpop.load_model();
    gwpop.load_vt();
    gwpop.load_posteriors();
    gwpop.create_likelihood();
    gwpop = gwpop

    SM = SmoothedTwoComponentPrimaryMassRatio(primary_mass_name='mass_1')
    R = gravpop.PowerLawRedshift(z_max=1.9)

    event_data = gwpop.likelihood.data.copy()
    event_data['prior'] = gwpop.likelihood.sampling_prior
    selection_data = {k:v for k,v in gwpop.selection.data.items()}

    gravpop_likelihood = PopulationLikelihood(models=[SM,R], 
                                                   event_data=event_data,
                                                   selection_data=selection_data,
                                                   analysis_time=selection_data['analysis_time'],
                                                   total_generated=selection_data['total_generated'])
    
    return gwpop.likelihood, gravpop_likelihood

In [41]:
gwpop_likelihood, gravpop_likelihood = create_likes()

11:23 bilby INFO    : Using SmoothedMassDistribution.
11:23 bilby INFO    : Created SmoothedMassDistribution with arguments {'mmin': 2, 'mmax': 100}
11:23 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
11:23 bilby INFO    : Created gwpopulation.models.redshift.PowerLawRedshift with arguments {'z_max': 1.9}
11:23 bilby INFO    : Using SmoothedMassDistribution.
11:23 bilby INFO    : Created SmoothedMassDistribution with arguments {'mmin': 2, 'mmax': 100}
11:23 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
11:23 bilby INFO    : Created gwpopulation.models.redshift.PowerLawRedshift with arguments {'z_max': 1.9}
11:23 bilby INFO    : Loading VT data from /Users/asadh/Downloads/o1+o2+o3_bbhpop_real+semianalytic-LIGO-T2100377-v2.hdf5.


In [69]:
loglikes = gravpop_likelihood.sampled_compute_log_weights(gravpop_likelihood.event_data, param)
per_events_log_bfs = jax.scipy.special.logsumexp(loglikes, axis=-1)
jnp.min(jnp.exp(2*per_events_log_bfs - jax.scipy.special.logsumexp(2*loglikes, axis=-1)), axis=-1)

Array(316.88477, dtype=float32)

In [70]:
gwpop_likelihood.parameters.update(param)
jnp.min(gwpop_likelihood.per_event_bayes_factors_and_n_effective()[1], axis=-1)

Array(316.88474, dtype=float32)

In [68]:
gravpop_likelihood.compute_event_min_N_eff(gravpop_likelihood.sampled_compute_log_weights(gravpop_likelihood.event_data, param))[1]

Array(316.88477, dtype=float32)

In [49]:
print(inspect.getsource(gwpop_likelihood.per_event_bayes_factors_and_n_effective_and_variances))

    def per_event_bayes_factors_and_n_effective_and_variances(self):
        """
        Called by `_compute_per_event_ln_bayes_factors` to compute the
        per event BFs, effective number of samples for each event's computed
        BF, and the associated uncertainty (variance) in the *ln* BF. Computes
        same qunatities as superclass function `_compute_per_event_ln_bayes_factors`
        but additionally provides the effective sample size.

        Returns
        -------
        per_event_bfs: array-like
            The BF per event, computed by reweighting single-event likelihood
            samples into the `hyper_prior` model.
        n_effectives: array-like
            The effective sample size for each Monte Carlo sum computation of the BFs.
            The BF is computed for each event, so this array has length n_events.
        variance: array-like
            The variances (uncertainties) in the ln BF per event.
        """
        weights = self.hyper_prior.prob(se

In [84]:
TT = TestAgainstGWPop()

10:30 bilby INFO    : Using SmoothedMassDistribution.
10:30 bilby INFO    : Created SmoothedMassDistribution with arguments {'mmin': 2, 'mmax': 100}
10:30 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
10:30 bilby INFO    : Created gwpopulation.models.redshift.PowerLawRedshift with arguments {'z_max': 1.9}
10:30 bilby INFO    : Using SmoothedMassDistribution.
10:30 bilby INFO    : Created SmoothedMassDistribution with arguments {'mmin': 2, 'mmax': 100}
10:30 bilby INFO    : Using PowerLawRedshift from gwpopulation.models.redshift.
10:30 bilby INFO    : Created gwpopulation.models.redshift.PowerLawRedshift with arguments {'z_max': 1.9}
10:30 bilby INFO    : Loading VT data from /Users/asadh/Downloads/o1+o2+o3_bbhpop_real+semianalytic-LIGO-T2100377-v2.hdf5.


In [86]:
TT.evaluate_gwpop_mass_model_at(param2), TT.evaluate_gravpop_mass_model_at(param2)

(Array([[-2.1898499, -7.691758 , -7.3970504, ..., -4.462355 , -8.406267 ,
         -2.9077847],
        [-4.6113043, -2.3372164, -2.0394418, ..., -2.739703 , -2.6363328,
         -2.8727822],
        [-4.872846 , -5.684493 , -2.4224682, ..., -7.483857 , -2.8173594,
         -2.2414067],
        ...,
        [-7.0089455, -5.5191   , -6.0251784, ..., -6.349538 , -2.3583875,
         -7.2968507],
        [-7.1197686, -7.661855 , -6.9311028, ..., -7.0771847, -6.81681  ,
         -7.7487793],
        [-6.966433 , -8.15437  , -7.213766 , ..., -8.120122 , -6.625331 ,
         -7.2415743]], dtype=float32),
 Array([[-2.1898499, -7.691758 , -7.3970504, ..., -4.4623556, -8.406267 ,
         -2.907785 ],
        [-4.6112967, -2.337211 , -2.0394115, ..., -2.7396796, -2.6363175,
         -2.8727658],
        [-4.872846 , -5.6844926, -2.422469 , ..., -7.4838567, -2.8173606,
         -2.241407 ],
        ...,
        [-7.008945 , -5.519101 , -6.025175 , ..., -6.3495364, -2.3583872,
         -7.2968507

In [87]:
TT.evaluate_gwpop_redshift_model_at(param2), TT.evaluate_gravpop_redshift_model_at(param2)

(Array([[-2.1799593 , -2.8006506 , -2.566938  , ..., -2.2161548 ,
         -2.5116687 , -2.3184078 ],
        [-4.567558  , -5.1628027 , -5.044173  , ..., -4.0300617 ,
         -3.988928  , -5.3443246 ],
        [-3.0181527 , -3.338162  , -3.031524  , ..., -3.5369866 ,
         -3.8482807 , -3.5539284 ],
        ...,
        [-0.8613874 , -0.44736767, -0.59904575, ..., -0.78943205,
         -0.73386425, -2.0654087 ],
        [-1.3912089 , -0.98895425, -0.4784534 , ..., -0.48603714,
         -0.3195701 , -1.739254  ],
        [-1.2963629 , -1.1313052 , -1.2221882 , ..., -1.0743494 ,
         -2.0336516 , -1.3593014 ]], dtype=float32),
 Array([[-2.1799622 , -2.8006427 , -2.566935  , ..., -2.2161613 ,
         -2.5116649 , -2.3183997 ],
        [-4.567533  , -5.1624994 , -5.044373  , ..., -4.0301194 ,
         -3.9889758 , -5.3444247 ],
        [-3.0181713 , -3.3381958 , -3.0315144 , ..., -3.5370238 ,
         -3.8482952 , -3.5538805 ],
        ...,
        [-0.86138743, -0.44736767, -0.5

In [89]:
HL.sampled_compute_log_weights(HL.event_data, param2)

Array([[ -8.190622 ,  -9.859154 ,  -8.221525 , ...,  -9.020756 ,
         -9.535853 ,  -7.5524683],
       [ -7.3783255,  -6.7408304,  -6.7934475, ...,  -7.004393 ,
         -7.695133 ,  -7.7830615],
       [ -7.325183 ,  -6.3512087,  -6.120516 , ...,  -5.560857 ,
         -8.900072 ,  -6.2334127],
       ...,
       [ -9.058701 ,  -8.015263 ,  -8.422426 , ...,  -8.149716 ,
         -8.106478 ,  -5.274376 ],
       [ -9.268319 ,  -9.315601 ,  -8.2897835, ...,  -7.956595 ,
         -8.730072 , -10.651072 ],
       [ -9.449632 ,  -7.7705708, -10.128664 , ...,  -3.3224092,
         -8.005751 ,  -5.1389556]], dtype=float32)

In [98]:
gwpop.likelihood.hyper_prior.parameters.update(param2)
jnp.log(gwpop.likelihood.hyper_prior.prob(gwpop.likelihood.data) / gwpop.likelihood.sampling_prior)

Array([[ -3.8342686 ,  -5.9112763 ,  -3.7089095 , ...,  -5.4233694 ,
         -5.811838  ,  -3.7597425 ],
       [  1.0770588 ,   1.1576973 ,   1.1903913 , ...,   1.6031827 ,
          0.81036717,  -0.29446906],
       [ -1.6835444 ,  -1.6131309 ,  -0.44074348, ...,  -0.07786553,
         -3.464705  ,  -0.58891666],
       ...,
       [ -8.912923  ,  -6.3197107 ,  -7.2942734 , ...,  -7.9638057 ,
         -6.504609  ,  -4.8025594 ],
       [ -9.663024  ,  -9.059093  ,  -8.439956  , ...,  -8.537923  ,
         -9.349231  , -11.157881  ],
       [ -6.7942266 ,  -7.867073  ,  -5.998494  , ...,  -1.3149095 ,
         -5.6662936 ,  -4.4511347 ]], dtype=float32)

In [99]:
inspect_it(HL.sampled_compute_log_weights)

    def sampled_compute_log_weights(self, data, params):
        return sum(self.log(model(data, params)) for model in self.models) - self.log(data["prior"])



In [100]:
gwpop.likelihood.sampling_prior

array([[1.28251023e-02, 1.92957228e-02, 1.09695755e-02, ...,
        2.73950808e-02, 2.41368212e-02, 2.25340048e-02],
       [2.12729649e-04, 3.71298735e-04, 3.40915165e-04, ...,
        1.82729075e-04, 2.02357305e-04, 5.59444278e-04],
       [3.54709560e-03, 8.75548953e-03, 3.41424151e-03, ...,
        4.15679010e-03, 4.35950220e-03, 3.53708253e-03],
       ...,
       [8.64349621e-01, 1.83497923e-01, 3.23630371e-01, ...,
        8.30347215e-01, 2.01519575e-01, 6.23867147e-01],
       [1.48394482e+00, 7.73748067e-01, 1.16203373e+00, ...,
        1.78841057e+00, 1.85736544e+00, 1.65998656e+00],
       [7.02701774e-02, 1.10131277e+00, 1.60801947e-02, ...,
        1.34324059e-01, 9.63797480e-02, 5.02669659e-01]])

In [102]:
HL.event_data["prior"]

Array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [76]:
min([1,2,3,4])

1